# Bagging, Boosting and Random Forest

Decision trees are already quite flexible but they are limited by how the splits are selected. The *greedy algorithm* of selecting the best split for the current node can affect the performance of nodes lower in the tree. Due to this some enhancements have been made to these to help in the bias-variance tradeoff.

Bagging stands for "*bootstrap aggregation*" which consists of using bootstrap sampling to grow various trees with a partial sample of the data to then average the results of each tree. This is particularly helpful to reduce variance in predictions. 

Boosting consists of 

Finally, Random forests are the growing of multiple trees but changing the variable selected to do the split every time so there is no bias from the *greedy algorithm*. We proceed to fit all 3 decision tree enhancements in this section.

# Libraries

In [27]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, ParameterGrid
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from yellowbrick.classifier import classification_report, confusion_matrix
from yellowbrick.classifier.rocauc import roc_auc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import roc_curve
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Import Data

In [22]:
X_train = pd.read_csv('0_X_train.csv', index_col='Id')
X_valid = pd.read_csv('1_X_valid.csv', index_col='Id')
X_test  = pd.read_csv('2_X_test.csv', index_col='Id')

y_train = pd.read_csv('0_y_train.csv', index_col='Id')
y_valid = pd.read_csv('1_y_valid.csv', index_col='Id')
y_test  = pd.read_csv('2_y_test.csv', index_col='Id')

# Build Bagged Tree

For each of the sections, we proceed to find the parameters that generate the best model by comparing their prediction accuracy scores. 

There are a series of parameters we can adjust. `min_samples_leaf` is the final number of observations in a determined terminal node. `n_estimators` is the amount of trees to be fitted for each round of model generation. `max_features` here is set to none which indicates the function can automatically decide which parameters to split on. This parameter will be modified in Random Forests.

In [3]:
bag_hyperparam_grid={'min_samples_leaf':[1, 3, 5, 7, 9,], "n_estimators":[500, 750, 1000, 1250, 1500, 2000,]}
bag = RandomForestClassifier(oob_score=True, max_features=None, criterion="log_loss",
                             warm_start=False, random_state=1, n_jobs=-2)
best_score=0.5

for g in ParameterGrid(bag_hyperparam_grid):
    bag.set_params(**g)
    bag.fit(X_train,y_train)
    # save if best
    if bag.oob_score_ > best_score:
        best_score = bag.oob_score_
        best_params = g

print(f"OOB: %0.5f" % best_score)
print("Best parameters:", best_params)

#Student note: takes over 25 mins to run

C:\Users\ficop\AppData\Local\Temp\ipykernel_31372\3809645309.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_31372\3809645309.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_31372\3809645309.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train,y_train)


KeyboardInterrupt: 

Once the best parameters have been found we can safe to a new variable and print the predictive accuracy of this model.

In [ ]:
#create the bagged tree
bag = RandomForestClassifier(min_samples_leaf = 3, n_estimators = 200,oob_score=True, 
                             max_features=None, criterion="log_loss",warm_start=False, 
                             random_state=1, n_jobs=-2)

#fit the model with training data
bag.fit(X_train, y_train)

C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\665392476.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bag.fit(X_train, y_train)


RandomForestClassifier(criterion='log_loss', max_features=None,
                       min_samples_leaf=3, n_estimators=200, n_jobs=-2,
                       oob_score=True, random_state=1)

In [ ]:
print("Train accuracy:", bag.oob_score_)
print("Validation accuracy:", bag.score(X_valid,y_valid))
print("Test accuracy:", bag.score(X_test,y_test))

Train accuracy: 0.8498244494095116
Validation accuracy: 0.8354430379746836
Test accuracy: 0.849590469099032


# Build Boosted Tree

For the boosted trees, an additional parameter is indicated. The learning rate ($\lambda$) adjusts the size of the tree at a certain rate for this it is called a *slow learner*. The `learning_rate` is  avalue between 0 and 1 and in finding the best model we indicate various values of this new parameter. Additionally, `max_depth` changes the number of splits done in the tree. As seen from regular trees, the number of nodes affects the prediction accuracy on all sets, so finding the most appropriate one is also important.

In [ ]:
boost_hyperparam_grid={'max_depth':[1, 3, 5, 7, 9,], 
                       "n_estimators":[500, 750, 1000, 1250, 1500, 2000,], 
                       "learning_rate":[0.001, 0.01, 0.1, 0.5, 1]}

boost = GradientBoostingClassifier(max_features=None, 
                             warm_start=False, random_state=1)

best_boost_score=0.5

for bg in ParameterGrid(boost_hyperparam_grid):
    boost.set_params(**bg)
    boost.fit(X_train,y_train)
    # save if best
    if boost.score(X_train, y_train) > best_boost_score:
        best_boost_score = boost.score(X_train, y_train)
        best_boost_params = bg

print(f"OOB: %0.5f" % best_boost_score)
print("Best parameters:", best_boost_params)

#Student note: takes over 50 mins to run

C:\Users\ficop\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ficop\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ficop\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\ficop\AppData\Roaming\Python\Python311\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array w

OOB: 1.00000
Best parameters: {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 1250}


As so we can see that the best combination tha best maximizes the accuracy is  `learning_rate`= 0.01, `max_depth`= 9, and `n_estimators`= 1250.

With this we train a new model and find the accuracy of the validation and test sets.

In [15]:
#boost = GradientBoostingClassifier(max_depth = best_boost_params["max_depth"], 
#                                   n_estimators = best_boost_params["n_estimators"],
#                                   learning_rate = best_boost_params["learning_rate"])

boost = GradientBoostingClassifier(max_depth = 9, 
                                   n_estimators = 1250,
                                   learning_rate = 0.01)

In [17]:
#show results
print("Train accuracy: ", boost_pipe.score(X_train, y_train))
print("Validation accuracy: ", boost_pipe.score(X_valid, y_valid))
print("Test accuracy: ", boost_pipe.score(X_test, y_test))

Train accuracy:  1.0
Validation accuracy:  0.8548026805658973
Test accuracy:  0.8607594936708861


For matters of comparison, we now run the best boosted model, as it has had a very good performance so far, with the predictors selected from Lasso variable selection to see if there is any improvement.

In [ ]:
#establish unrequired variable names
cols_lasso = ['banner_views_old', 'days_elapsed_old', 'X3', 'marital_divorced', 'job_entrepreneur', 'job_freelance',
             'job_housekeeper', 'job_technology', 'job_unemployed']

#create new partitions with corresponding lasso variables
x_train_mod = X_train.drop(columns=cols_lasso)
x_valid_mod = X_valid.drop(columns=cols_lasso)
x_test_mod = X_test.drop(columns=cols_lasso)

In [4]:
#fit the model on new training data
boost.fit(x_train_mod, y_train)

#show results of model
print(boost.score(x_train_mod, y_train))
print(boost.score(x_valid_mod, y_valid))
print(boost.score(x_test_mod, y_test))

NameError: name 'boost' is not defined

As we can see there is no improvement from full variable boosting and the variable selection. Now we proceed to the same procedure with Random Forests.

# Random Forest


The new parameter to choose in Random Forest is the number of features. This refers to the number of variables used in the random selection for growing new trees. To establish an appropopriate amount, rule of thumb is to start with the $\sqrt{p}$, where $p$ is the number of predictors in the data set.

In [38]:
default_max_features_param = np.sqrt(X_train.shape[1])
default_max_features_param

5.5677643628300215

As in this case its close to 5.5 we go two units below 5 and 2 units above 6 to have a decent set to choose from.

In [34]:
#set hyperparameter grid
rf_hyperparam_grid={"max_features":[3, 4, 5, 6, 7, 8],
                 'min_samples_leaf':[1, 3, 5, 7, 9],
                 "n_estimators":[500, 750, 1000, 1250, 1500, 2000]}

#instantiate the randfortest
rfc = RandomForestClassifier(criterion = "log_loss", oob_score=True, warm_start=False, random_state=1, n_jobs=-2)
best_score=0.5

#loop over parameters. running duration 24mins.
for rfg in ParameterGrid(rf_hyperparam_grid):
    rfc.set_params(**rfg)
    rfc.fit(X_train,y_train)
    # save if best
    if rfc.oob_score_ > best_score:
        best_rfc_score = rfc.oob_score_
        best_rfc_params = rfg

#print best results
print(f"OOB: %0.5f" % best_rfc_score)
print("Best parameters:", best_rfc_params)

#Student note: takes over 30 mins to run

C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\2324745643.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\2324745643.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\2324745643.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\2324745643.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train,y_train)
C:\U

OOB: 0.84376
Best parameters: {'max_features': 7, 'min_samples_leaf': 9, 'n_estimators': 2000}


We can see that the best combination is `max_features` = 7, `min_samples_leaf`= 9, and `n_estimators`= 2000. Now the final model is saved to a variable.

In [28]:
rfc = RandomForestClassifier(n_estimators=best_rfc_params['max_features'], 
                             max_features=best_rfc_params['max_features'],
                             min_samples_leaf=best_rfc_params['min_samples_leaf'], 
                             criterion="log_loss",oob_score=True, warm_start=False, random_state=1)


C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\2296077635.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


In [39]:
# fit the model
rfc.fit(X_train, y_train)

#predict value
y_valid_pred = rfc.predict(X_valid)
y_test_pred = rfc.predict(X_test)

C:\Users\ficop\AppData\Local\Temp\ipykernel_22560\3517693396.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


In [37]:
print("Train accuracy:", rfc.oob_score_)
print("Validation accuracy:", rfc.score(X_valid,y_valid))
print("Test accuracy:", rfc.score(X_test,y_test))

Train accuracy: 0.8450367060325567
Validation accuracy: 0.827252419955324
Test accuracy: 0.8510796723752793


Finally, we can see that despite the result being better than the original tree, it did not improve upon the boosting results obtained previously.